---
# 환경 설정
---

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers==4.3.0
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 3.2 MB/s 
     |████████████████████████████████| 3.3 MB 60.2 MB/s 
     |████████████████████████████████| 880 kB 50.6 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=bc9e8b3a9fef2967f6810f3e6de7c26ec64edc068d017fa3a1bee0e4205c4df9
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# pip install wandb

In [ ]:
pip install transformers-interpret

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 41 kB 308 kB/s 
     |████████████████████████████████| 1.4 MB 5.3 MB/s 
     |████████████████████████████████| 793 kB 50.6 MB/s 
     |████████████████████████████████| 1.6 MB 49.8 MB/s 
  Attempting uninstall: ipython
    Found existing installation: ipython 7.9.0
    Uninstalling ipython-7.9.0:
      Successfully uninstalled ipython-7.9.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires ipython~=7.9.0, but you have ipython 7.34.0 which is incompatible.


In [ ]:
# import wandb

# wandb.login()
# wandb.init()

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [ ]:
import pandas as pd 
from sklearn.model_selection import train_test_split
import re
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score

In [ ]:
#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
# GPU 사용
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Kobert/TAPT_Model_Save")

bertmodel = AutoModel.from_pretrained("/content/drive/MyDrive/Kobert/TAPT_Model_Save")

Some weights of BertModel were not initialized from the model checkpoint at /content/drive/MyDrive/Kobert/TAPT_Model_Save and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


---
# 학습용 데이터셋 전처리
---

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Kobert/data/final_emotional_dataset.csv')
del data['Unnamed: 0']
data = data.drop_duplicates(subset = 'content')
data = data.dropna()
data = data.reset_index(drop=True)
data

,content,label
0,접근성 좋아요,1
1,친절함은 좋았으나 4인이 식사를 하는데 밑반찬까지 한그릇씩만 나와서 먹기에 불편 했...,0
2,고기는 맛있는데 서버들 몇명은 불친절하고 주문도 제대로 못받음,0
3,모든 면에서 강추합니다 주변에서 14년 살았는데 젤 좋습니다,1
4,사장님 직원분들 너무 친절하시구 음식도 맛있어요,1
...,...,...
102540,사람마다 틀리겠지만 불친절하네 말투하며 접수할맘도 안생기네 ㅎㅎ 따른데 갈란다,0
102541,이집 소개해서 욕먹은적없는 곱창 좀 먹어본분들이 가시면 좋습니다 양마니와 오발탄을 ...,1
102542,메뉴 구성도 좋고 맛있고 직원분들도 친절하고 깔끔하고 사장님 친절하시고 퍼펙트합니다,1
102543,먹어본 공차중에 버블 제일 딱딱하고 적음 최악,0


In [ ]:
data[data['content'].str.contains('배달 빠르고 맛있게 먹었네요 무 양이 적어요')]

,content,label
52250,배달 빠르고 맛있게 먹었네요 무 양이 적어요,1


In [ ]:
data2 = pd.read_csv('/content/drive/MyDrive/Kobert/data/total_train_0831.csv')
data2 = data2[['review','taste']]
del data2['taste']
data2

,review
0,양도 좋고 맛도 좋습니다 배달도 빨라요
1,양이많고 배달도 빠르네요 전 국물떡볶이가 맛있었답니다
2,너무 맛있어용 서비스로 주신 분모자도 감사합니다 아래 도어 번호를 안 써서 못들어 ...
3,사진을 드럽게 못 찍긴 했지만 배달 양 맛 싹 다 넘모 죠습니다
4,배달 완전 정확하게 오고 맛도 양도 넘 좋아요 맛있게 잘 먹었습니다
...,...
29659,할인까지 받아서 정말 맛나게 머것어요
29660,오랜만에 자메이카통다리 주문 밥이랑 먹기 좋았어요
29661,황금올리브 닭다리는 언제나 옳아요 맛있어요
29662,맛도좋구배달도빠르네요


In [ ]:
data2[(data2['review'].str.contains('배달')) & data2['review'].str.contains('빠름')]

,review
93,양 적절함 맛 적절함 배달 빠름
128,양상추 시들 감자 시들 소스부족 퍽퍽 배달만빠름
178,맛집임 가격도 저렴 배달도 빠름 치킨집 여기 정착
243,맛있어요 배달도 항상 빠름
256,예전보단 버거가 작아진 느낌이지만 그래도 맛나요 맛있게 잘 먹었어요 배달 빠름
...,...
25636,배달 엄청 빠름 켄터키 구이 맛있어어
25720,맛있습니다용 배달빠름
25856,맛도좋고배달이으음청빠름
25966,kfc 맛있어요 배달도 빠름


In [ ]:
data2[(data2['review'].str.contains('배달')) & data2['review'].str.contains('빠르')]

,review
1,양이많고 배달도 빠르네요 전 국물떡볶이가 맛있었답니다
6,감자랑 떡 양이 미쳤어요 배달도 정말 빠르고 껍질이 바삭해서 좋아요 식후감
8,배달도 빠르고 양도 많네요 맛있게 먹었습니다
9,배달 빠르고 맛있게 먹었네요 무 양이 적어요
12,양도 많고 배달도 빠르고 맛있었습니다
...,...
29590,조금 맵지만 잘 먹었습니다 배달도 빠르고 따끈하네요
29591,배달 빠르고 양도 맛도 훌륭합니다 잘먹었습니다
29628,맛잇어요 배달도 빠르고
29656,배달도 빠르고 맛있어서 아이들이 좋아합니다


----

In [ ]:
import numpy as np 

total_data_text = list(data['content'])
# 텍스트데이터 문장길이의 리스트를 생성한 후
num_tokens = [len(tokens) for tokens in total_data_text]
num_tokens = np.array(num_tokens)
# 문장길이의 평균값, 최대값, 표준편차를 계산해 본다. 
print('문장길이 평균 : ', np.mean(num_tokens))
print('문장길이 최대 : ', np.max(num_tokens))
print('문장길이 표준편차 : ', np.std(num_tokens))

# 예를들어, 최대 길이를 (평균 + 2*표준편차)로 한다면,  
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
maxlen = int(max_tokens)
print('pad_sequences maxlen : ', maxlen)
print('전체 문장의 {}%가 maxlen 설정값 이내에 포함됩니다. '.format(np.sum(num_tokens < max_tokens) / len(num_tokens)))

문장길이 평균 :  73.12752450143839
문장길이 최대 :  2025
문장길이 표준편차 :  94.56406201708111
pad_sequences maxlen :  262
전체 문장의 0.9617046174850066%가 maxlen 설정값 이내에 포함됩니다. 


In [ ]:
train_data, test_data = train_test_split(data, test_size = 0.2 ,random_state= 632)
print(train_data.shape)
print(test_data.shape)

(82036, 2)
(20509, 2)


---
# 모델
---

In [ ]:
class BERTDataset(Dataset): 
  def __init__(self, dataset, bert_tokenizer, max_len):
    
    self.sentences = [bert_tokenizer(i, truncation=True, return_token_type_ids=True, padding="max_length",
                                     max_length = max_len, return_tensors = 'pt') for i in dataset['content']]
    self.max_len = max_len
    self.label = [np.int64(i) for i in dataset['label']]
  
  def __getitem__(self,i):
    self.input_ids = self.sentences[i]['input_ids']
    self.attention_mask_token = self.sentences[i]['attention_mask']
    self.token_type_ids = self.sentences[i]['token_type_ids']

    return self.input_ids, self.attention_mask_token, self.token_type_ids, self.label[i]

  def __len__(self):
    return (len(self.label))

In [ ]:
# 하이퍼파라미터 세팅
max_len = 262
batch_size = 16
warmup_ratio = 0.1
num_epochs = 2
max_grad_norm = 1
log_interval = 300
learning_rate =  5e-7

In [ ]:
data_train = BERTDataset(train_data, tokenizer, max_len)
data_test = BERTDataset(test_data, tokenizer, max_len)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, shuffle=True, batch_size=batch_size, num_workers=2, drop_last=True)
test_dataloader = torch.utils.data.DataLoader(data_test, shuffle=True, batch_size=batch_size, num_workers=2, drop_last=True)

In [ ]:
# dataloader test 
for a,b,c,d in train_dataloader:
  print(a)
  print(b)
  print(c)
  print(d)
  print('='*100)
  print('input_ids: ',a.shape)
  print('attention_mask: ',b.shape)
  print('token_type_ids: ',c.shape)
  print('label: ',d.shape)
  break


tensor([[[    2,  1316,  2117,  ...,  2116, 10159,     3]],

        [[    2,  1378,  1586,  ...,     0,     0,     0]],

        [[    2,  9801,  5545,  ...,     0,     0,     0]],

        ...,

        [[    2,  3869, 22883,  ...,     0,     0,     0]],

        [[    2, 15781,  2170,  ...,     0,     0,     0]],

        [[    2, 11130,  4867,  ...,     0,     0,     0]]])
tensor([[[1, 1, 1,  ..., 1, 1, 1]],

        [[1, 1, 1,  ..., 0, 0, 0]],

        [[1, 1, 1,  ..., 0, 0, 0]],

        ...,

        [[1, 1, 1,  ..., 0, 0, 0]],

        [[1, 1, 1,  ..., 0, 0, 0]],

        [[1, 1, 1,  ..., 0, 0, 0]]])
tensor([[[0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0]],

        ...,

        [[0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0]]])
tensor([0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0])
input_ids:  torch.Size([16, 1, 262])
attention_mask:  torch.Size([16, 1, 262])
token_type_i

In [ ]:
# wandb.init(project = 'kobert_emotional',
#                  entity="glad415",
#                  config = {
#                      "hidden_size" : 768,
#                      "max_len" : max_len,
#                      "batch_size" : batch_size,
#                      "learning_rate" : learning_rate,
#                      "epochs" : num_epochs,
#                      "optimizer" : 'SGD',
#                      "loss" : "BCEloss",
#                  })

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 ################# 클래스 수 조정 ##################
                 num_classes=2,   
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.dense = nn.Linear(hidden_size , num_classes)

        self.dropout = nn.Dropout(p=dr_rate)
    
    def forward(self, input_ids, attention_mask, token_type_ids):

        out = self.bert(input_ids = input_ids.long(), token_type_ids = token_type_ids.long(), attention_mask = attention_mask.to(device))
        out = self.dropout(out.pooler_output)

        return self.dense(out)
    
    ############################# Frezee #############################
    # def freeze_bert_encoder(self):
    #     for param in self.bert.parameters():
    #         param.requires_grad = False
    
    # def unfreeze_bert_encoder(self):
    #     for param in self.bert.parameters():
    #         param.requires_grad = True
    ##################################################################

In [ ]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

# optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum = 0.9)
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    total_f1 = 0.0
    for batch_id, (input_ids, attention_mask_token, token_type_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        
        optimizer.zero_grad()
        
        input_ids = input_ids.long().to(device)
        input_ids = input_ids.squeeze(1)

        token_type_ids = token_type_ids.long().to(device)
        token_type_ids = token_type_ids.squeeze(1)
        
        attention_mask_token= attention_mask_token.squeeze(1)

        label = label.long().to(device)

        out = model(input_ids, attention_mask_token, token_type_ids)

        # Loss
        ################################################################################################
        loss = loss_fn(out, label)

        loss.backward() 
        ################################################################################################

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)

        optimizer.step()
        scheduler.step()  # Update learning rate schedule

        # Accuracy
        ################################################################################################
        preds = torch.argmax(out,dim = 1).flatten()
        accuracy = (preds == label).cpu().numpy().mean()

        train_acc += accuracy 
        ################################################################################################

        # F1_Score
        ################################################################################################
        F1_Score = f1_score(preds.cpu().data, label.cpu(), average = 'macro')
        
        total_f1 +=  F1_Score
        ################################################################################################

        if batch_id % log_interval == 0:

            print(f"epoch {e+1} batch id {batch_id+1} loss {loss.data.cpu().numpy():.3f}")
            print(f'train acc {train_acc / (batch_id+1):.3f} f1_score:{total_f1 / (batch_id+1):.3f}')
            print('#'*80)

            # WandB
            #############################################################################################################################
            # wandb.log({'interval_Loss': loss.data.cpu().numpy(),
            #            'interval_Train_acc' : train_acc, 'interval_F1_Score' : total_f1})
            #############################################################################################################################

    print(f"epoch {e+1} epoch_Train_Accuracy {train_acc / (batch_id+1):.3f}, epoch_F1_score {total_f1 / (batch_id+1):.3f}")
    
    # WandB
    #############################################################################################################################
    # wandb.log({'epoch_Train_Accuracy' : train_acc / (batch_id+1), 'epoch_F1_score' : total_f1 / (batch_id+1)})
    #############################################################################################################################

    model.eval()
    with torch.no_grad():
      test_f1 = 0.0
      test_acc = 0.0

      for batch_id, (input_ids, attention_mask_token, token_type_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
          
          input_ids = input_ids.long().to(device)
          input_ids = input_ids.squeeze(1)

          token_type_ids = token_type_ids.long().to(device)
          token_type_ids = token_type_ids.squeeze(1)
          
          attention_mask_token= attention_mask_token.squeeze(1)

          label = label.long().to(device)

          out = model(input_ids, attention_mask_token, token_type_ids)

          # Test_Accuracy
          ################################################################################################    
          preds = torch.argmax(out,dim = 1).flatten()
          accuracy = (preds == label).cpu().numpy().mean()

          test_acc += accuracy
          ################################################################################################ 

          # Test_F1_Score
          ################################################################################################
          F1_Score = f1_score(preds.cpu().data, label.cpu(), average = 'macro')
          
          test_f1 +=  F1_Score
          ################################################################################################

      print('-'*40,'Test_Value','-'*40)
      print(f"test_Accuracy {test_acc / (batch_id+1):.3f}, test_F1_score {test_f1/ (batch_id+1):.3f}")
      print('-'*92)

      # wandb.log({'Test_acc' : test_acc / (batch_id+1), 'Test_F1_Score' : test_f1/ (batch_id+1)})

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


  0%|          | 0/5127 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.669
train acc 0.500 f1_score:0.492
################################################################################
epoch 1 batch id 301 loss 0.672
train acc 0.578 f1_score:0.561
################################################################################
epoch 1 batch id 601 loss 0.506
train acc 0.661 f1_score:0.645
################################################################################
epoch 1 batch id 901 loss 0.400
train acc 0.738 f1_score:0.725
################################################################################
epoch 1 batch id 1201 loss 0.153
train acc 0.785 f1_score:0.774
################################################################################
epoch 1 batch id 1501 loss 0.341
train acc 0.815 f1_score:0.805
################################################################################
epoch 1 batch id 1801 loss 0.258
train acc 0.835 f1_score:0.826
########################################################################

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:71: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/1281 [00:00<?, ?it/s]

---------------------------------------- Test_Value ----------------------------------------
test_Accuracy 0.945, test_F1_score 0.942
--------------------------------------------------------------------------------------------


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


  0%|          | 0/5127 [00:00<?, ?it/s]

epoch 2 batch id 1 loss 0.022
train acc 1.000 f1_score:1.000
################################################################################


KeyboardInterrupt: ignored

In [ ]:
# wandb.finish()

In [ ]:
# torch.save(model.state_dict(), '/content/drive/MyDrive/Kobert/model/koBERT_emotional_dict_0913.pt')

In [ ]:
# model = BERTClassifier(bertmodel).to(device)
# model.load_state_dict(torch.load('/content/drive/MyDrive/Kobert/model/koBERT_emotional_dict_0913.pt', map_location=device))

In [ ]:
def softmax(x):
  exp_x = np.exp(x)
  result = exp_x / np.sum(exp_x)

  return result

def predict(predict_sentence):

    dataset_another = pd.DataFrame({'content': predict_sentence, 'label' : 0 }, index = [0])

    another_test = BERTDataset(dataset_another,tokenizer, max_len)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=1, num_workers=5)
    
    model.eval()

    for input_ids, attention_mask_token, token_type_ids, label in test_dataloader:

          input_ids = input_ids.long().to(device)
          input_ids = input_ids.squeeze(1)

          token_type_ids = token_type_ids.long().to(device)
          token_type_ids = token_type_ids.squeeze(1)
          
          attention_mask_token= attention_mask_token.squeeze(1)

          out = model(input_ids, attention_mask_token, token_type_ids)
          negative, positive = softmax(out[0].detach().cpu().numpy())

          print(f'긍정 점수: {positive:.3f}')
          print(f'부정 점수: {negative:.3f}')
          

---
# 결과값 테스트
---

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
predict('맛이 없다')

긍정 점수: 0.009
부정 점수: 0.991


In [ ]:
predict('서비스가 좋아서 다시 가고 싶어요')

긍정 점수: 0.995
부정 점수: 0.005


In [ ]:
predict('돈가스 맛집이라해서 가봤는데 양이 너무 적어요')

긍정 점수: 0.017
부정 점수: 0.983


In [ ]:
predict('음식은 겉바속촉 매장은 깔끔')

긍정 점수: 0.085
부정 점수: 0.915


In [ ]:
predict('가까워서 빠른 배달이 좋아요 상하이 맛있구요')

긍정 점수: 0.997
부정 점수: 0.003


In [ ]:
predict('맛 양 배달 모두 만족합니다')

긍정 점수: 0.996
부정 점수: 0.004


In [ ]:
predict('생각보다 빨리 와서 좋았어요 ㅎㅎ')

긍정 점수: 0.995
부정 점수: 0.005


In [ ]:
predict('다 좋은데 피자가 찌그러져서 왔네요')

긍정 점수: 0.127
부정 점수: 0.873


In [ ]:
predict('사업 번창하세요')

긍정 점수: 0.989
부정 점수: 0.011


In [ ]:
predict('콜라가 거의 다 넘쳤어요 조금 흐른 정도가 아니라 쏟아진 수준이라 감튀도 축축해지고 ㅜㅜ')

긍정 점수: 0.006
부정 점수: 0.994


In [ ]:
predict('기사님 너무 친절하시고 감자도 꾹꾹 담아서 가득 주셔서 배부르게 먹었어요 콜라도 뚜껑 이중으로 해주셔서 하나도 안 새고 왔습니당')

긍정 점수: 0.995
부정 점수: 0.005


In [ ]:
predict('다른집 들렸다 온것 같아요')

긍정 점수: 0.109
부정 점수: 0.891


In [ ]:
predict('배달은 빠른데 음식이 맛없어')

긍정 점수: 0.005
부정 점수: 0.995


In [ ]:
predict('음식이 맛없는데 배달만 빠르네')

긍정 점수: 0.005
부정 점수: 0.995


In [49]:
predict('배달이 빠름')

긍정 점수: 0.013
부정 점수: 0.987


In [ ]:
predict('음식이 식어서와서 화가났지만 한입먹고 맛있어서 화가 풀렸어요')

긍정 점수: 0.972
부정 점수: 0.028


In [106]:
model.save_pretrained('/content/drive/MyDrive/Kobert/file.pt')

AttributeError: ignored

In [51]:
from transformers_interpret import SequenceClassificationExplainer
cls_explainer = SequenceClassificationExplainer(model,tokenizer)
word_attributions = cls_explainer("배달이 빠름")

AttributeError: ignored

In [103]:
a = {'가' :2, "나":3, "다":5}

In [104]:
[a.pop(key, None) for key in ['가', '다']]

[2, 5]

In [105]:
a

{'나': 3}